In [39]:
import itertools
import numpy as np

In [40]:
def generate_boolean_functions(n):
    inputs = list(itertools.product([0, 1], repeat=n))
    num_functions = 2**(2**n)
    functions = []
    for i in range(num_functions):
        output = [int(x) for x in f"{i:0{2**n}b}"]
        functions.append(list(zip(inputs, output)))
    return functions

In [41]:
def relu(Z):
    return np.maximum(0, Z)

def relu_derivative(Z):
    return np.where(Z > 0, 1, 0)

In [42]:
def forward_propagate(weights, biases, X):
    activations = [X]
    for l in range(len(weights)):
        Z = np.dot(activations[-1], weights[l]) + biases[l]
        A = relu(Z)
        activations.append(A)
    return activations[-1]

In [43]:
def backward_propagate(weights, biases, X, y, output):
    L = len(weights)
    d_weights = [np.zeros_like(w) for w in weights]
    d_biases = [np.zeros_like(b) for b in biases]
    error = output - y
    delta = error
    
    for l in reversed(range(L)):
        d_weights[l] = np.dot(X.T if l == 0 else relu_derivative(np.dot(X, weights[l-1])).T, delta)
        d_biases[l] = np.sum(delta, axis=0, keepdims=True)
        if l > 0:
            delta = np.dot(delta, weights[l].T) * relu_derivative(np.dot(X, weights[l-1]))
    
    return d_weights, d_biases

In [44]:
def create_ann(num_inputs, num_outputs, layers, hidden_layer_size=4):
    np.random.seed(0)
    weights = []
    biases = []
    for i in range(layers):
        input_dim = num_inputs if i == 0 else hidden_layer_size
        output_dim = hidden_layer_size if i < layers - 1 else num_outputs
        weights.append(np.random.randn(input_dim, output_dim))
        biases.append(np.zeros((1, output_dim)))
    return weights, biases

In [45]:
def train_ann(ann, X, y, learning_rate, epochs):
    weights, biases = ann
    for _ in range(epochs):
        output = forward_propagate(weights, biases, X)
        delta_weights, delta_biases = backward_propagate(weights, biases, X, y, output)
        weights = [w - learning_rate * dw for w, dw in zip(weights, delta_weights)]
        biases = [b - learning_rate * db for b, db in zip(biases, delta_biases)]
    return weights, biases

def test_ann(ann, X, y):
    weights, biases = ann
    output = forward_propagate(weights, biases, X)
    predictions = np.round(output)
    accuracy = np.mean(predictions == y)
    return accuracy

In [46]:
def is_learnable(function, max_layers, learning_rate, epochs, threshold=0.95):
    X, y = zip(*function)
    X = np.array(X)
    y = np.array(y).reshape(-1, 1)
    for L in range(1, max_layers + 1):
        ann = create_ann(len(X[0]), 1, L)
        trained_ann = train_ann(ann, X, y, learning_rate, epochs)
        if test_ann(trained_ann, X, y) >= threshold:
            return True
    return False

In [47]:
def count_learnable_functions(n, max_layers=1, learning_rate=0.1, epochs=1000, threshold=0.95):
    functions = generate_boolean_functions(n)
    learnable_counts = {L: 0 for L in range(1, max_layers + 1)}
    
    for func in functions:
        for L in range(1, max_layers + 1):
            if is_learnable(func, L, learning_rate, epochs, threshold):
                learnable_counts[L] += 1
                break
    
    return learnable_counts

In [49]:
n_values = [1,2,3]
for n in n_values:
    learnable_counts = count_learnable_functions(n)
    print(f"For n={n}:")
    for L, count in learnable_counts.items():
        print(f"  Learnable with {L} layers neurons: {count}/{2**(2**n)}")

For n=1:
  Learnable with 1 layers neurons: 4/4
For n=2:
  Learnable with 1 layers neurons: 14/16
For n=3:
  Learnable with 1 layers neurons: 85/256
